In [1]:
import numpy as np
from ROAR.utilities_module.occupancy_map import OccupancyGridMap
from pathlib import Path
import logging
from pathlib import Path
from ROAR_Sim.configurations.configuration import Configuration as CarlaConfig
from ROAR_Sim.carla_client.carla_runner import CarlaRunner
from ROAR.agent_module.pure_pursuit_agent import PurePursuitAgent
from ROAR.configurations.configuration import Configuration as AgentConfig
from ROAR.agent_module.michael_pid_agent import PIDAgent
from ROAR.agent_module.rl_prep_agent import RLPrepAgent
from ROAR.agent_module.special_agents.waypoint_generating_agent import WaypointGeneratigAgent
from ROAR.agent_module.special_agents.recording_agent import RecordingAgent
import argparse
from misc.utils import str2bool
from ROAR.utilities_module.data_structures_models import Transform, Location
import cv2

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [7]:
#file = open("C://Users//roar//Desktop//ROAR_record//ROAR_record//data//output//border_waypoints//right_side_major_map_waypoints.txt")
file = open(".//data//waypoints//major_map_waypoints2.txt")
full_list = []
for r in file:
    cur_list = []
    seg = r.rstrip().split(',')
    for item in seg:
        cur_list.append(float(item))
    full_list.append(cur_list)
waypoints = np.array(full_list)
#waypoints[:, 3:] = [0, 0, 0]
#waypoints[:, :3] += np.array([50, -8, 295])
waypoints[0]

array([2599.19995117,  104.74610138, 4372.60009766,    0.        ,
          0.        ,   10.00061798])

In [8]:
len(waypoints)
print(waypoints[0])

[2599.19995117  104.74610138 4372.60009766    0.            0.
   10.00061798]


In [9]:
agent_config = AgentConfig.parse_file(Path("./ROAR/configurations/carla/carla_agent_configuration.json"))
agent = PIDAgent(vehicle=None, agent_settings=agent_config)

loadmap = OccupancyGridMap(agent)
#loadmap.load_from_file(Path("C://Users//roar//Desktop//ROAR_record//ROAR_record//data//output//occupancy_map//occu_map.npy"))
loadmap.load_from_file(Path("data//cleaned_occ_map//berkeley_major_cleaned_occu_map.npy"))

In [10]:
local_transform = Transform.from_array(waypoints[0])
#local_map = loadmap.get_map(transform = local_transform)
local_map = loadmap.get_map(transform = local_transform, view_size=(100, 100), boundary_size=(20, 20), show_agent= True)
cv2.imshow("local_map", local_map) # uncomment to show occu map
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
count = 0
k = 10
for i in range(len(waypoints)//k):
    #count += 1
    #print(waypoints[k*i])
    loadmap.update_to_one(np.array([waypoints[i*k][:3]]))
    transform = Transform.from_array(waypoints[k*i])
    local_map = loadmap.get_map(transform = transform)
    # x, y = points[0], points[2]
    cv2.imshow("local_map", local_map) # uncomment to show occu map
    cv2.waitKey(1)
    #cv2.destroyAllWindows()
cv2.destroyAllWindows()
print(count)

KeyboardInterrupt: 